In [6]:
import numpy as np
import matplotlib.pyplot as pl
from pykalman import KalmanFilter, UnscentedKalmanFilter, AdditiveUnscentedKalmanFilter
from simglucose.simulation.user_interface import simulate
from simglucose.simulation.env import T1DSimEnv
from simglucose.controller.basal_bolus_ctrller import BBController
from simglucose.sensor.cgm import CGMSensor
from simglucose.actuator.pump import InsulinPump
from simglucose.patient.t1dpatient import T1DPatient
from simglucose.simulation.scenario_gen import RandomScenario
from simglucose.simulation.scenario import CustomScenario
from simglucose.simulation.sim_engine import SimObj, sim, batch_sim
from datetime import timedelta
from datetime import datetime
import ipywidgets as widgets
import random
from IPython.display import display


# ----------Simulation
# Code adapted from https://github.com/jxx123/simglucose

# Random Scenario
now = datetime.now()
start_time = datetime.combine(now.date(), datetime.min.time())
# Specify results saving path
path = './results'
# Create a simulation environment
patient = T1DPatient.withName('adolescent#006')
sensor = CGMSensor.withName('Dexcom', seed=1)
pump = InsulinPump.withName('Insulet')
scenario = RandomScenario(start_time=start_time, seed=random.randrange(1,200))
env = T1DSimEnv(patient, sensor, pump, scenario)
# Create a controller
controller = BBController()

# Put them together to create a simulation object
s1 = SimObj(env, controller, timedelta(days=2), animate=False, path=path)
results1 = sim(s1)

rnd = np.random.RandomState(0)
n_timesteps = 961
x = np.linspace(0, 3 * np.pi, n_timesteps)

#Raw Data
#print(results1.columns)
#print(results1)



# print(len(results.index))
# x = results1.loc[:,"Time"]
actual = results1.loc[:,"BG"]
observation = results1.loc[:,"CGM"]
print(len(observation))







# transition_cov=1000*np.eye(1) 

# https://medium.com/dataman-in-ai/kalman-filter-explained-4d65b47916bf

alpha = 0.1

#My try on the book's statement on page 195

transition_matr = [1 for i in range(n_timesteps)] 
avgBG = np.average(observation)
for t in range(n_timesteps-1):
    
    l = transition_matr[t-1] + alpha * ((avgBG - 120)/120)
    transition_matr[t] = l
    
# print(transition_matr)


def transition_function(state, noise):
    l = state + alpha * ((avgBG - 120)/120)
    return state

def observation_function(state, noise):
    return state

def additive_transition_function(state):
   
    return state

def additive_observation_function(state):
    
    return state

kf = KalmanFilter(additive_transition_function, additive_observation_function,
                #transition_matrices = 1,    # The value for At. => 1
                # observation_matrices = 1,   # The value for Ht. -> 1
                initial_state_mean = 0,       #  Initial value. It will converge to the true state value. -> 0
                initial_state_covariance = 1, # Sigma value Qt  -> 1
              observation_covariance=1,     # Sigma value Rt    -> 1           
             transition_covariance=0.032) # 0.032

akf = AdditiveUnscentedKalmanFilter(
    additive_transition_function, additive_observation_function,
    initial_state_mean = 0,       #  Initial value. It will converge to the true state value.
                   initial_state_covariance = 1, # Sigma value Qt 
                  observation_covariance=1,     # Sigma value Rt              
                  transition_covariance=0.022
)


states_pred_smooth_mean = kf.smooth(observation)[0]
states_pred_filter_mean = kf.filter(observation)[0]

states_pred_smooth_mean_kfu = akf.smooth(observation)[0]
states_pred_filter_mean_kfu = akf.filter(observation)[0]


# -------- Plotting ---------------

# Measurement
pl.figure(figsize=(16, 6))
pl.scatter(x,observation, marker='x', color='b', label='observations')
position_line = pl.plot(x, actual,markersize=2,
                        linestyle='-', marker='o', color='g',
                        label='actual')
pl.xlabel('time')
pl.ylabel('BG')
pl.show()


# Filter
pl.figure(figsize=(16, 6))
obs_scatter = pl.scatter(x, observation, marker='x', color='b',
                         label='observations', alpha=0.5)
position_line = pl.plot(x, states_pred_filter_mean,markersize=2,
                        linestyle='-', marker='o', color='red',
                        label='filter')
position_line = pl.plot(x, actual,markersize=2,
                        linestyle='-', marker='o', color='g',
                        label='actual')
pl.legend(loc='lower left',fontsize=14)
pl.xlim(xmin=0, xmax=x.max())
pl.xlabel('time')
pl.show()

#Smooth
pl.figure(figsize=(16, 6))
obs_scatter = pl.scatter(x, observation, marker='x', color='b',
                         label='observations', alpha=0.5)
position_line = pl.plot(x, states_pred_smooth_mean,markersize=2,
                        linestyle='-', marker='o', color='red',
                        label='smooth')
position_line = pl.plot(x, actual,markersize=2,
                        linestyle='-', marker='o', color='g',
                        label='actual')
pl.legend(loc='lower left',fontsize=14)
pl.xlim(xmin=0, xmax=x.max())
pl.xlabel('time')
pl.show()


# Filter
pl.figure(figsize=(16, 6))
obs_scatter = pl.scatter(x, observation, marker='x', color='b',
                         label='observations', alpha=0.5)
position_line = pl.plot(x, states_pred_filter_mean_kfu,markersize=2,
                        linestyle='-', marker='o', color='red',
                        label='filter')
position_line = pl.plot(x, actual,markersize=2,
                        linestyle='-', marker='o', color='g',
                        label='actual')
pl.legend(loc='lower left',fontsize=14)
pl.xlim(xmin=0, xmax=x.max())
pl.xlabel('time')
pl.show()

#Smooth
pl.figure(figsize=(16, 6))
obs_scatter = pl.scatter(x, observation, marker='x', color='b',
                         label='observations', alpha=0.5)
position_line = pl.plot(x, states_pred_smooth_mean_kfu,markersize=2,
                        linestyle='-', marker='o', color='red',
                        label='smooth')
position_line = pl.plot(x, actual,markersize=2,
                        linestyle='-', marker='o', color='g',
                        label='actual')
pl.legend(loc='lower left',fontsize=14)
pl.xlim(xmin=0, xmax=x.max())
pl.xlabel('time')
pl.show()

#simulate()
#results = batch_sim(s, parallel=True)
#print(results)

Process ID: 7404
Simulation starts ...
Simulation Completed!
961


TypeError: unsupported operand type(s) for *: 'function' and 'float'